In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')


In [5]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [7]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)


In [18]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
loader = WebBaseLoader('https://pytorch.org/docs/stable/nn.html')
documents = loader.load()

In [21]:
documents

[Document(metadata={'source': 'https://pytorch.org/docs/stable/nn.html', 'title': 'torch.nn — PyTorch 2.6 documentation', 'language': 'en'}, page_content='\n\n\n  \n\n\n\ntorch.nn — PyTorch 2.6 documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Learn\n              \n\n\nGet Started\nRun PyTorch locally or get started quickly with one of the supported cloud platforms\n\n\nTutorials\nWhats new in PyTorch tutorials\n\n\nLearn the Basics\nFamiliarize yourself with PyTorch concepts and modules\n\n\nPyTorch Recipes\nBite-size, ready-to-deploy PyTorch code examples\n\n\nIntro to PyTorch - YouTube Series\nMaster PyTorch basics with our engaging YouTube tutorial series\n\n\n\n\n\n\n\n                Ecosystem\n              \n\n\nTools\nLearn about the tools and frameworks in the PyTorch Ecosystem\n\n\nCommunity\nJoin the PyTorch developer community to contribute, learn, and get your questions answered\n\n\nForums\nA p

In [22]:
text_docs = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=500).split_documents(documents)
text_docs

[Document(metadata={'source': 'https://pytorch.org/docs/stable/nn.html', 'title': 'torch.nn — PyTorch 2.6 documentation', 'language': 'en'}, page_content="torch.nn — PyTorch 2.6 documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Learn\n              \n\n\nGet Started\nRun PyTorch locally or get started quickly with one of the supported cloud platforms\n\n\nTutorials\nWhats new in PyTorch tutorials\n\n\nLearn the Basics\nFamiliarize yourself with PyTorch concepts and modules\n\n\nPyTorch Recipes\nBite-size, ready-to-deploy PyTorch code examples\n\n\nIntro to PyTorch - YouTube Series\nMaster PyTorch basics with our engaging YouTube tutorial series\n\n\n\n\n\n\n\n                Ecosystem\n              \n\n\nTools\nLearn about the tools and frameworks in the PyTorch Ecosystem\n\n\nCommunity\nJoin the PyTorch developer community to contribute, learn, and get your questions answered\n\n\nForums\nA place to discuss 

In [23]:
vector_store = FAISS.from_documents(text_docs, OpenAIEmbeddings())
vector_store

In [24]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1180cb7d0>, search_kwargs={})

In [25]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, 'Containers', 'Search what are Containers in pytorch and what are the types')
retriever_tool.name

'Containers'

In [26]:
tools=[wiki,arxiv,retriever_tool]

In [27]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='Llama-3.3-70b-Versatile')

In [28]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [33]:
from langchain.agents import  AgentExecutor

agent_executor = AgentExecutor(tools=tools, agent=agent, verbose=True)

In [34]:
agent_executor.invoke({
  "input":"1706.03762"
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, base
Invoking: `wikipedia` with `{'query': 'Transformer (machine learning) model'}`


Page: Transformer (deep learning architecture)
Summary: The transformer is a deep learning architecture that was developed by researchers at Google and is based on the multi-head attention mechanism, which was proposed in the 2017 paper "Attention Is All You Need". Text is converted to numerical representations called tokens

KeyboardInterrupt: 